extract symptom from original datasets

In [ ]:
with open("/content/train.txt","r") as f:
  with open("/content/train-all-Disease.txt","w") as wf:
    lines = f.readlines()
    i = 0
    start = end = 0
    tag = 0
    for i in range(len(lines)):
      line = lines[i]
      if(line=="\n"):
        end = i
        if(tag==1): # loop write the lines: from start to end
          for j in range(start,end+1):
            wf.write(lines[j])
          tag = 0
        start = end + 1
      else:
        line = line.strip("\n")
        if(line.endswith("DEVICE")):
          tag = 1
          #print(line)


随机抽取250个

In [ ]:
import random
random.sample(range(0,100),10)

[43, 83, 91, 60, 11, 9, 13, 3, 62, 55]

In [ ]:
with open("/content/train-all-Disease.txt","r") as f:
  lines = f.readlines()
  count = 0
  for line in lines:
    if(line=="\n"):
      count+=1

index = random.sample(range(0,count),5)

with open("/content/train-all-Disease.txt","r") as f:
  with open("/content/train-5-Disease.txt","w") as wf:
    lines = f.readlines()
    i = 0
    start = end = 0
    c = 0
    for i in range(len(lines)):
      line = lines[i]
      if(line=="\n"):
        c += 1
        end = i
        if(c in index):
          for j in range(start,end+1):
            wf.write(lines[j])
        start = end + 1

In [ ]:
count

26044

In [ ]:
with open("/content/train-5-Disease.txt","r") as f:
  lines = f.readlines()
  count = 0
  for line in lines:
    if(line=="\n"):
      count+=1

In [ ]:
count

10

分割线，下面需要的是，extract training data中的所有disease！

再分割一下，后面的scripts主要就是从这里分开的

step 1: Extract all concepts from training data

In [ ]:
with open("/content/train.txt","r") as f:
  lines = f.readlines()
  i = 0
  tmp = []
  entity = set()
  for i in range(len(lines)-1):
    line = lines[i].strip("\n")
    next_line = lines[i+1].strip("\n")
    if(line.endswith("B-Disease")):
      tmp = []
      if(next_line.endswith("I-Disease")):     # need to change I-GENE
        tmp.append(line.split(" ")[0])
        while(next_line.endswith("I-Disease")):
          tmp.append(next_line.split(" ")[0])
          i+=1
          next_line = lines[i+1].strip("\n")
        phrase = ""
        for j in range(len(tmp)-1):
          #wf.write(tmp[j] + " ")
          phrase += tmp[j]
          phrase += " "
        #wf.write(tmp[j+1] + "\n")
        phrase += tmp[j+1]
        #phrase += "\n"
        entity.add(phrase)
      else:
        entity.add(line.split(" ")[0])
        #wf.wirte(line + "\n")
print(entity)
with open("/content/train-disease-entities.txt","w") as wf:
  for ent in entity:
    print(ent)
    wf.write(ent + "\n")

{'granulocytopenia', 'hypertensive', 'atrial tachycardia', 'intraventricular conduction abnormalities', 'toxicity', 'intra-Hisian block', 'hypotensive', 'hyperalgesia'}
granulocytopenia
hypertensive
atrial tachycardia
intraventricular conduction abnormalities
toxicity
intra-Hisian block
hypotensive
hyperalgesia


Step 2: write the scripts for search for all related concepts

1. save all urls
2. 每个entity要建立一个文件，以entity命名的
3. 根据每个url，得到一个json文件
4. parse json文件，得到name，和cui
5. name可以直接生成每个entity对应的related concepts文件，但是只要10个就够了。
6. cui保存起来做字典，供后面查parents、children使用。



In [ ]:
#apikey = "7e7e253a-1168-4530-b8b4-24cb7daaa701"
#api_url = "https://uts-ws.nlm.nih.gov/rest/search/current"

#with open("/content/train-disease-entities.txt","r") as f:
  #with open("/content/train-disease-url-concepts.txt","w") as wf:
    #lines = f.readlines()
    #for line in lines:
      #string = line.strip("\n")
      #url = api_url + "?apiKey=" + apikey + "&string=" + string
      #wf.write(url+"\n")

In [ ]:
!mkdir /content/related-concepts/

In [ ]:
!rm -r /content/related-concepts/

每个entity要建立一个文件，以entity命名的

根据每个url，得到一个json文件

parse json文件，得到name

name可以直接生成每个entity对应的related concepts文件，但是只要10个就够了。


In [ ]:
import requests
import json

apikey = "7e7e253a-1168-4530-b8b4-24cb7daaa701"
api_url = "https://uts-ws.nlm.nih.gov/rest/search/current"

related_concepts_path = "/content/related-concepts/"

with open("/content/train-disease-entities.txt","r") as f:
    lines = f.readlines()
    for line in lines:
      #print(line)
      string = line.strip("\n")
      url = api_url + "?apiKey=" + apikey + "&string=" + string
      response = requests.get(url)

      data = json.loads(response.content)

      with open(related_concepts_path + line.strip("\n") + ".txt","a") as wf:
        i = 1
        for concept in data['result']['results']:
          if(i<=10):
            #print(concept['name'])
            wf.write(concept['name']+"\n")
          i+=1


再分割，尝试一下解析

In [ ]:
import requests
import json

# 解析出来的网址
url = 'https://uts-ws.nlm.nih.gov/rest/search/current?apiKey=7e7e253a-1168-4530-b8b4-24cb7daaa701&string=granulocytopenia'

# 获取网址的内容
response = requests.get(url)

# 将响应内容转换为 Python 对象
data = json.loads(response.content)

#print(data['result']['results'])
i = 1
for concept in data['result']['results']:
  if(i<=10):
    print(concept['name'])
  i+=1

# 将 Python 对象保存为 json 文件
#with open('/content/data.json', 'w') as f:
    #json.dump(data, f)


Parents and Children

得到given concept的cui

根据cui去找它的parent和children

也是最多只要10个就行

In [ ]:
!mkdir /content/parents/

In [ ]:
!rm -r /content/parents/

得到given concept的cui

In [ ]:
import requests
import json

apikey = "7e7e253a-1168-4530-b8b4-24cb7daaa701"
api_url = "https://uts-ws.nlm.nih.gov/rest/search/current"

related_concepts_path = "/content/related-concepts/"

with open("/content/train-disease-entities.txt","r") as f:
  with open("/content/train-disease-uis.txt","w") as wf:
    lines = f.readlines()
    for line in lines:
      #print(line)
      string = line.strip("\n")
      url = api_url + "?apiKey=" + apikey + "&string=" + string + "&returnIdType=code&sabs=SNOMEDCT_US"
      response = requests.get(url)

      #print(string)
      data = json.loads(response.content)
      if(len(data['result']['results'])!=0):
        wf.write(string + "\t" + data['result']['results'][0]['ui']+"\n")
        #print(data['result']['results'])

根据cui去找它的parent和children

也是最多只要10个就行

In [ ]:
import requests
import json
import os

apikey = "7e7e253a-1168-4530-b8b4-24cb7daaa701"
api_url = "https://uts-ws.nlm.nih.gov/rest/content/current/source/SNOMEDCT_US/"

parents_children_path = "/content/parents/"

with open("/content/train-disease-uis.txt","r") as f:
  lines = f.readlines()
  for line in lines:
    #print(line)
    string = line.strip("\n").split("\t")[1]
    #print(string)
    url = api_url + string + "/parents?apiKey=" + apikey
    #print(url)
    response = requests.get(url)

    data = json.loads(response.content)

    with open(parents_children_path + line.strip("\n").split("\t")[0] + ".txt","a") as wf:
      i = 1
      for concept in data['result']:
        if(i<=10):
          #print(concept['name'])
          wf.write(concept['name']+"\n")
        i+=1

    url = api_url + string + "/children?apiKey=" + apikey
    response = requests.get(url)

    data = json.loads(response.content)
    #print(url,data)

    file_name = parents_children_path + line.strip("\n").split("\t")[0] + ".txt"
    if(os.path.exists(file_name)):
      with open(file_name,"r") as rf:
        lines = rf.readlines()
        i = len(lines)+1
        #print(file_name,i)
    else:
      i = 1
    with open(file_name,"a") as wf:
      #if(exists, i = lens; else: i = 0)
      if('result' in data):
        for concept in data['result']:
          if(i<=10):
            #print(concept['name'])
            wf.write(concept['name']+"\n")
          i+=1


In [ ]:
url = "https://uts-ws.nlm.nih.gov/rest/content/current/source/SNOMEDCT_US/9468002/parents?apiKey=7e7e253a-1168-4530-b8b4-24cb7daaa701"

response = requests.get(url)

data = json.loads(response.content)

print(data['result'][0]['name'])

with open('/content/data.json', 'w') as f:
    json.dump(data, f)

Closed fracture of upper limb


Siblings

得到given concept的parent的cui

根据cui去找它的children

也是最多只要10个就行

In [ ]:
!mkdir /content/siblings/

In [ ]:
!rm -r /content/siblings/

得到given concept的parent的cui

In [ ]:
import requests
import json
import os

apikey = "7e7e253a-1168-4530-b8b4-24cb7daaa701"
api_url = "https://uts-ws.nlm.nih.gov/rest/content/current/source/SNOMEDCT_US/"

with open("/content/train-disease-uis.txt","r") as f:
  with open("/content/train-disease-parents-uis.txt","w") as wf:
    lines = f.readlines()
    for line in lines:
      #print(line)
      string = line.strip("\n").split("\t")[1]
      #print(string)
      url = api_url + string + "/parents?apiKey=" + apikey
      #print(url)
      response = requests.get(url)

      data = json.loads(response.content)

      if('result' in data):
        wf.write(line.strip("\n").split("\t")[0] + "\t" + data['result'][0]['ui']+"\n")

根据cui去找它的children

也是最多只要10个就行

In [ ]:
import requests
import json
import os

apikey = "7e7e253a-1168-4530-b8b4-24cb7daaa701"
api_url = "https://uts-ws.nlm.nih.gov/rest/content/current/source/SNOMEDCT_US/"

parents_children_path = "/content/siblings/"

with open("/content/train-disease-parents-uis.txt","r") as f:
  lines = f.readlines()
  for line in lines:
    #print(line)
    string = line.strip("\n").split("\t")[1]
    #print(string)
    url = api_url + string + "/children?apiKey=" + apikey
    response = requests.get(url)

    data = json.loads(response.content)
    #print(url,data)

    file_name = parents_children_path + line.strip("\n").split("\t")[0] + ".txt"
    i = 1
    with open(file_name,"a") as wf:
      if('result' in data):
        for concept in data['result']:
          if(i<=10):
            #print(concept['name'])
            wf.write(concept['name']+"\n")
          i+=1


zip three folders

In [ ]:
!tar -zcvf bc5cdr-parents.tar.gz /content/parents/
!tar -zcvf bc5cdr-related-concepts.tar.gz /content/related-concepts/
!tar -zcvf bc5cdr-siblings.tar.gz /content/siblings/

calculate the frequency of text， copy from EntLM scripts

In [ ]:
with open("/content/train.txt") as f:
  text = f.readlines()
  DEVICE = []
  HEALTHPLAN = []
  BIOID = []
  IDNUM = []
  for line in text:
    if(line!="\n"):
      new_line = line.strip("\n").split(" ")
      if(new_line[1]!="O"):
        #print(line)
        #SpecificDisease, Modifier, DiseaseClass
        if(new_line[1].endswith("DEVICE")):
          DEVICE.append(new_line[0])
        elif(new_line[1].endswith("HEALTHPLAN")):
          HEALTHPLAN.append(new_line[0])
        elif(new_line[1].endswith("BIOID")):
          BIOID.append(new_line[0])
        elif(new_line[1].endswith("IDNUM")):
          IDNUM.append(new_line[0])

In [ ]:
import string
punctuation_string = string.punctuation
#print(punctuation_string)

DEVICE_new = []

for word in DEVICE:
  for i in punctuation_string:
    word = word.replace(i,"")
  #print(word)
  DEVICE_new.append(word.lower())

In [ ]:
import collections

frequency = collections.Counter(DEVICE_new)
print(frequency)

Counter({'193062closure': 1, '358892': 1, '8068103total': 1, '17722gnp': 1, '30058catheters': 1, '56520yog': 1, '9ys7ez': 1})


Transform data from txt to json,  copy from EntLM scripts

In [ ]:
with open("/content/5shot-1.txt","r") as f:
  with open("/content/5shot-1.json","a") as wf:
    text = f.readlines()
    word = []
    label = []
    for line in text:
      if(line=="\n"):
        wf.write('{"text": [')
        count = 0
        for w in word:
          count+=1
          if(count!=len(word)):
            wf.write('"'+w+'", ')
          else:
            wf.write('"'+w+'"')
        wf.write('], "label": [')
        count = 0
        for l in label:
          count+=1
          if(count!=len(label)):
            wf.write('"'+l+'", ')
          else:
            wf.write('"'+l+'"')
        wf.write(']}' +"\n")
        word = []
        label = []
      else:
        new_line = line.strip("\n").split(" ")
        word.append(new_line[0])
        label.append(new_line[1])